In [37]:
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms


In [38]:
#hyper parameters
batch_size=100
epochs=5
learning_rate=0.001

In [44]:
train_dataset= datasets.MNIST(root='./data',train=True,download=True,transform=transforms.ToTensor())
test_dataset=datasets.MNIST(root='./data',train=False,download=False,transform=transforms.ToTensor())

#create loader
train_loader=torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_loader=torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)


2.6%

100.0%


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw


100.0%
9.9%


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw



100.0%
100.0%


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



In [45]:
train_dataset

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [46]:
for images,labels in train_loader:
    print(images.shape)
    print(labels)
    break
    

torch.Size([100, 1, 28, 28])
tensor([1, 2, 9, 3, 8, 2, 1, 3, 1, 8, 0, 8, 4, 9, 6, 1, 9, 0, 2, 0, 9, 8, 1, 2,
        4, 5, 2, 0, 8, 6, 9, 2, 7, 0, 0, 6, 0, 8, 9, 8, 4, 0, 7, 1, 1, 7, 9, 1,
        8, 4, 0, 4, 8, 7, 5, 4, 1, 7, 3, 8, 7, 6, 9, 1, 7, 8, 4, 1, 6, 6, 7, 9,
        5, 8, 0, 9, 2, 5, 5, 8, 2, 3, 3, 5, 7, 0, 6, 9, 1, 1, 3, 4, 4, 0, 8, 0,
        3, 2, 3, 8])


In [50]:
class Block(nn.Module):
    """Some Information about Block"""
    def __init__(self):
        super(Block, self).__init__()
        self.con1=nn.Conv2d(in_channels=1, out_channels=16,bias=False, kernel_size=3, padding=1, stride=1)
        self.batch1=nn.BatchNorm2d(16)
        self.relu1=nn.ReLU(inplace=True)
        self.con2=nn.Conv2d(16, 1,bias=False, kernel_size=3, padding=1, stride=1)
        self.batch2=nn.BatchNorm2d(1)
        self.Linear1=nn.Linear(1*28*28,100)
        self.drop = nn.Dropout(p=0.5, inplace=False)
        self.Linear2=nn.Linear(100,10)
        
        


    def forward(self, x):
        residual=x.clone()
        x=self.con1(x)
        x=self.batch1(x)
        x=self.relu1(x)
        x=self.con2(x)
        x=self.batch2(x)
        x+=residual
        x=self.relu1(x)

        
        x=x.view(-1,1*28*28)
        x=self.Linear1(x)
        x=self.drop(x)
        x=self.Linear2(x)
        return x

In [51]:
train_loss=[]
training_accuracy=[]
test_loss=[]
test_accuracy=[]
model=Block()
loss_fn=nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [53]:
#training phase

for epoch in range(epochs):
    correct=0
    iterations=0
    iter_loss=0
    model=model.train()
    for i,(images,labels) in enumerate(train_loader):
        preds=model(images)
        loss=loss_fn(preds,labels)
        iter_loss+=loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _,predicted_labels=torch.max(preds,1)
        correct+=(predicted_labels==labels).sum()
        iterations+=1
    train_loss.append(iter_loss/iterations)
    training_accuracy.append(100* correct/len(train_dataset))
    #testing phase
    correct=0
    iter_loss=0
    iterations=0
    model=model.eval()
    for i,(images,labels) in enumerate(test_loader):
        preds=model(images)
        loss=loss_fn(preds,labels)
        iter_loss+=loss.item()
        _,predicted_labels=torch.max(preds,1)
        correct+=(predicted_labels==labels).sum()
        iterations+=1
    test_loss.append(iter_loss/iterations)
    test_accuracy.append(100* correct/len(test_dataset))
    print('epoch : {}/{}, training loss={},training accuracy={:.3f},test_loss={},test_accuracy={:.3f}'.format(epoch+1,epochs,train_loss[-1],training_accuracy[-1],
    test_loss[-1],test_accuracy[-1]))
        
    

epoch : 1/5, training loss=0.39696081570039193,training accuracy=88.103,test_loss=0.2286604313738644,test_accuracy=92.800
epoch : 2/5, training loss=0.26303749372561774,training accuracy=92.290,test_loss=0.1843459930177778,test_accuracy=94.300
epoch : 3/5, training loss=0.22701700985431672,training accuracy=93.168,test_loss=0.16983365898020567,test_accuracy=94.710
epoch : 4/5, training loss=0.20618672652790943,training accuracy=93.793,test_loss=0.1616393835004419,test_accuracy=95.360
epoch : 5/5, training loss=0.19448520687098306,training accuracy=94.197,test_loss=0.1564936307677999,test_accuracy=95.350


In [56]:
img=test_dataset[50][0].resize_((1,1,28,28))
label=test_dataset[50][1]
prediction=model(img)
_,predicted_label=torch.max(prediction,1)
print("true label is ",label)
print("predicted label is",predicted_label.item())

true label is  6
predicted label is 6
